# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)
## Requirements:

A description of the problem and a discussion of the background.<br>
A description of the data and how it will be used to solve the problem.<br>
For the second week, the final deliverables of the project will be:

A link to your Notebook on your Github repository, showing your code.<br>
A full report consisting of all of the following components:<br>
Introduction where you discuss the business problem and who would be interested in this project.<br>
Data where you describe the data that will be used to solve the problem and the source of the data.<br>
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.<br>
Results section where you discuss the results.<br>
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.<br>
Conclusion section where you conclude the report.<br>
Your choice of a presentation or blogpost.<br>

## A description of the problem and a discussion of the background.

A decission has to be made in order to select the city to place a restaurant. Oakland, Emeryville, and San Diego have been selected for the search. Very good reasons have to be exposed in order to atract potential investors in the food business. The type of the restaurant, the city, and the location in the city, are key factors to get success in the business.

The scope of this project is to accurately predict the acceptance that a new restaurant can expect based on the type of cuisine and the location in the cities selected. linear and logistic regressions are used to find which method is better for the prediction.

## A description of the data and how it will be used to solve the problem.

Initial dataframe used is named 'raw_dataset', it has the information required to make the analysis.<br>

Foursquare is used to retrive city coordinates, and to obtain the URLs with the raw data in JSON. From each URL: 'name', 'categories', 'latitude', 'longitude', and'id' columns are taken for each city, so a city column is also included.<br>
Restaurants in a 1000km radius from the coordinates provided by the geolocator, will be analized. Cleaning is performed to remove noisy data, and getting only restaurant valid data provided by Foursquare. 'likes' data is important for decission making. Only valuable information is pulled in order to have a strong analysis.<br>
'id' column is used in order to pull the 'likes' and include the information in the dataframe. 

## Methodology section

Both linear and logistic regression are used to train and test the data.<br>
Linear regression is used to predict the number of 'likes' a new restaurant in this region will acquire. Sci-Kit Learn is used for this stage.<br>
Logistic regression is used as the classification method.<br>
Since, binning is used when classifying by number of 'likes', multinomial logistic regression is used to perform the analysis.<br>
Although the ranges are discrete categories, they can be considered ordinal in nature.<br>
The logistic regression is specified as being both multinomial and ordinal.

### Code

In [28]:
import pandas as pd 
import numpy as np 
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
import matplotlib.pyplot as plt
import pylab as pl
import itertools
import warnings
from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
from sklearn import linear_model
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')
print('All libraries have been imported')

All libraries have been imported


### Retrieving Foursquare City Coordinates:

In [29]:
address1 = 'Oakland, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))

address2 = 'Emeryville, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))

address3 = 'San Diego, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location3 = geolocator.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address3, latitude3, longitude3))

The geograpical coordinate of Oakland, California are 37.8044557, -122.2713563.
The geograpical coordinate of Emeryville, California are 37.8314089, -122.2865266.
The geograpical coordinate of San Diego, California are 32.7174202, -117.1627728.


### Foursquare Credentials:

In [30]:
CLIENT_ID = 'R3JTQTGYXBW0HQCG5BYPSW3AAOLL3KOUTOUATGMPZSQ01LXB' 
CLIENT_SECRET = 'K1YBPNPGZ1NJIYKS2ILN41SHNVSK4SGGSL3IXCD0SUA4Y3SQ' 
VERSION = '20201212' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)


# create URLs
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)

print(url1, url2, url3)

Your credentails:
CLIENT_ID: R3JTQTGYXBW0HQCG5BYPSW3AAOLL3KOUTOUATGMPZSQ01LXB
CLIENT_SECRET:K1YBPNPGZ1NJIYKS2ILN41SHNVSK4SGGSL3IXCD0SUA4Y3SQ
https://api.foursquare.com/v2/venues/explore?&client_id=R3JTQTGYXBW0HQCG5BYPSW3AAOLL3KOUTOUATGMPZSQ01LXB&client_secret=K1YBPNPGZ1NJIYKS2ILN41SHNVSK4SGGSL3IXCD0SUA4Y3SQ&v=20201212&ll=37.8044557,-122.2713563&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=R3JTQTGYXBW0HQCG5BYPSW3AAOLL3KOUTOUATGMPZSQ01LXB&client_secret=K1YBPNPGZ1NJIYKS2ILN41SHNVSK4SGGSL3IXCD0SUA4Y3SQ&v=20201212&ll=37.8314089,-122.2865266&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=R3JTQTGYXBW0HQCG5BYPSW3AAOLL3KOUTOUATGMPZSQ01LXB&client_secret=K1YBPNPGZ1NJIYKS2ILN41SHNVSK4SGGSL3IXCD0SUA4Y3SQ&v=20201212&ll=32.7174202,-117.1627728&radius=1000&limit=100
